In [ ]:
import csv
from selenium import webdriver
import time
import re
from parsel import Selector
import os
import glob

key_words = [

    'id',
    'property_location_country',
    'property_location_region',
    'property_location_province',
    'property_location_district',
    'property_location_locality',
    'property_location_postalCode',
    'property_location_street',
    'property_location_number',
    'property_location_latitude',
    'property_location_longitude',
    'property_type',
    'property_subtype',
    'property_bedroomCount',
    'property_showerRoomCount',
    'property_bathroomCount',
    'property_netHabitableSurface',
    'property_roomCount',
    'property_building_condition',
    'property_building_constructionYear',
    'property_building_facadeCount',
    'property_propertyCertificates',
    'property_constructionPermit_floodZoneType',
    'property_constructionPermit_hasPossiblePriorityPurchaseRight',
    'property_constructionPermit_totalBuildableGroundFloorSurface',
    'property_energy_heatingType',
    'property_energy_hasHeatPump',
    'property_energy_hasPhotovoltaicPanels',
    'property_energy_hasThermicPanels',
    'property_energy_performance',
    'property_kitchen_type',
    'property_land_hasGasWaterElectricityConnection',
    'property_land_surface',
    'property_toiletCount',
    'property_hasSwimmingPool',
    'property_fireplaceExists',
    'property_hasTerrace',
    'property_terraceSurface',
    'property_terraceOrientation',
    'transaction_sale_price',
    'publication_creationDate',
    'transaction_certificates_carbonEmission',
    'transaction_certificates_epcScore',
    'transaction_sale_cadastralIncome',     

def extract_urls(url):
    try:
        ######################## WARNING ################################
        PATH = "/opt/chromedriver"  # TO CHANGE SET THE PATH FOR YOUR DRIVER
        #################################################################

        #### SCRAPPING ALL THE URLs #######

        driver = webdriver.Chrome(PATH)

        # An implicit wait tells WebDriver to poll the DOM for a certain amount of time when trying to find any element (or elements) not immediately available.
        # For the documentation: https://selenium-python.readthedocs.io/waits.html
        driver.implicitly_wait(10)

        driver.get(url)

        # For the documentation: https://parsel.readthedocs.io/en/latest/usage.html
        sel = Selector(text=driver.page_source)  # Initialize the Selector

        xpath_houses = '//*[@id="main-content"]/li//h2//a/@href'  # for a sheet cheat: https://devhints.io/xpath

        page_url = sel.xpath(xpath_houses).getall()  # returns a list with all results

        driver.close()
        return page_url
        ######### PERSISTING URLs IN THE CSV ########
    except:
        raise Exception
    
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import csv

# from backward_app.utils import parse_page_functions #Personnal function

def immoweb_parse(url):
    page = requests.get(url)
    if page.status_code == 200:
        soup    = BeautifulSoup(page.content, "lxml")
        script  = soup.find_all("script", type="text/javascript")
        raw     = (
                script[2]
                .decode_contents()
                .lstrip("\nwindow.classified = )")
                .rstrip(";\n        ")
                )
        # print(raw)
        dictionary = flatten_json(json.loads(raw))
        if dictionary['price_type'] != 'residential_sale': 
            return None
        if dictionary["transaction_sale_price"] == None:
            return None
        return dictionary
    else:
        return None

def flatten_json(json_raw):
    out = {}
    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x
    flatten(json_raw)
    return out

for url in page_url:
    dico_json = immoweb_parse(url)
    dic_temp = {}

    if dico_json == None:
                continue

    for key_word in key_words:
        try:
            dic_temp[key_word] = dico_json[key_word]
        except:
            dic_temp[key_word] = None           

    with open("info_house.csv", "a+") as file:
        csv_writer = csv.DictWriter(file, fieldnames = key_words)
        csv_writer.writerow(dic_temp)
